<a href="https://colab.research.google.com/github/jamelof23/Finance/blob/main/100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**next action dont go sec filing code , find api that can work for ozk too**



sec filing directly

In [ ]:
# Install required libraries
!pip install pandas openpyxl beautifulsoup4 requests lxml tqdm

import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from google.colab import files
from tqdm.notebook import tqdm

# Function to get CIK for a given symbol
def get_cik_map():
    # SEC provides a mapping file for tickers to CIKs
    url = 'https://www.sec.gov/files/company_tickers.json'
    headers = {'User-Agent': 'Jamil jamil.arbas@gmail.com'}
    response = requests.get(url, headers=headers)
    data = response.json()
    cik_map = {}
    for item in data.values():
        symbol = item['ticker']
        cik = str(item['cik_str']).zfill(10)
        cik_map[symbol.upper()] = cik
    return cik_map

def parse_form(cik, accession_number, headers):
    try:
        # Remove leading zeros from CIK and dashes from accession number for the directory
        cik_int = int(cik.lstrip('0'))
        accession_number_no_dashes = accession_number.replace('-', '')
        base_url = f"https://www.sec.gov/Archives/edgar/data/{cik_int}/{accession_number_no_dashes}"
        index_url = f"{base_url}/{accession_number}-index.html"
        print(f"Fetching index page: {index_url}")

        response = requests.get(index_url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve index page at {index_url}")
            return None

        index_content = response.content
        soup = BeautifulSoup(index_content, 'html.parser')

        # Find the table containing the document list
        documents_table = soup.find('table', {'class': 'tableFile', 'summary': 'Document Format Files'})
        if not documents_table:
            print("Could not find the documents table on the index page")
            return None

        # Initialize variable for XML file link
        xml_file_link = None

        # Iterate over table rows to find the XML file
        for row in documents_table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) >= 3:
                description = cells[1].get_text(strip=True).lower()
                document_link_tag = cells[2].find('a')
                if document_link_tag:
                    document_link = document_link_tag['href']
                    document_filename = document_link.split('/')[-1]
                    if document_filename.lower().endswith('.xml'):
                        xml_file_link = 'https://www.sec.gov' + document_link
                        print(f"Found XML file link: {xml_file_link}")
                        break

        if not xml_file_link:
            print("Could not find the XML file link in the index page")
            return None

        # Download and parse the XML file
        print(f"Fetching XML file: {xml_file_link}")
        response = requests.get(xml_file_link, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve XML file at {xml_file_link}")
            return None

        xml_content = response.content
        soup = BeautifulSoup(xml_content, 'xml')

        transactions = []
        reporting_owner_tag = soup.find('reportingOwner')
        if reporting_owner_tag and reporting_owner_tag.find('rptOwnerName'):
            reporting_owner = reporting_owner_tag.find('rptOwnerName').text
            print(f"Reporting owner: {reporting_owner}")
        else:
            reporting_owner = ''
            print("No reporting owner found")

        non_derivative_table = soup.find('nonDerivativeTable')
        if non_derivative_table:
            print("Non-derivative transactions found")
            for row in non_derivative_table.find_all('nonDerivativeTransaction'):
                transaction = {}
                transaction['ownerName'] = reporting_owner
                transaction['securityTitle'] = row.find('securityTitle').get_text(strip=True) if row.find('securityTitle') else ''
                transaction['transactionDate'] = row.find('transactionDate').find('value').text if row.find('transactionDate') else ''
                transaction['transactionCode'] = row.find('transactionCoding').find('transactionCode').text if row.find('transactionCoding') else ''
                transaction['transactionShares'] = row.find('transactionShares').find('value').text if row.find('transactionShares') else ''
                transaction['transactionPricePerShare'] = row.find('transactionPricePerShare').find('value').text if row.find('transactionPricePerShare') else ''
                transaction['sharesOwnedFollowingTransaction'] = row.find('postTransactionAmounts').find('sharesOwnedFollowingTransaction').find('value').text if row.find('postTransactionAmounts') else ''
                transactions.append(transaction)
            print(f"Parsed {len(transactions)} transactions")
        else:
            print("No non-derivative transactions found in the filing")
            return None

        return transactions
    except Exception as e:
        print(f"Error parsing form with accession number {accession_number}: {e}")
        return None


# Function to fetch insider transactions from SEC EDGAR
def fetch_insider_transactions(cik_map):
    insider_data = []
    symbols_with_data = []
    symbols_without_data = []
    headers = {'User-Agent': 'Your Name your.email@example.com'}
    total = len(cik_map)
    requests_made = 0
    rate_limit_per_second = 10  # SEC requests no more than 10 requests per second
    try:
        for symbol, cik in tqdm(cik_map.items(), total=total):
            try:
                # Print the symbol being processed
                print(f"\nProcessing symbol {symbol} ({requests_made + 1}/{total})")
                # Fetch company submissions
                url = f'https://data.sec.gov/submissions/CIK{cik}.json'
                response = requests.get(url, headers=headers)
                data = response.json()

                # Extract filings
                filings = data.get('filings', {}).get('recent', {})
                df = pd.DataFrame(filings)

                if not df.empty:
                    # Filter for Forms 3, 4, 5 (insider transactions)
                    insider_forms = df[df['form'].isin(['3', '4', '5'])]

                    if not insider_forms.empty:
                        data_found = False
                        # Limit to the last 5 filings to reduce processing time
                        insider_forms = insider_forms.head(5)
                        for index, row in insider_forms.iterrows():
                            accession_number = row['accessionNumber']
                            filing_date = row['filingDate']
                            form_type = row['form']

                            # Parse the Form
                            form_data = parse_form(cik, accession_number, headers)
                            if form_data:
                                data_found = True
                                for transaction in form_data:
                                    transaction['symbol'] = symbol
                                    transaction['filingDate'] = filing_date
                                    transaction['formType'] = form_type
                                    insider_data.append(transaction)
                        if data_found:
                            print(f"Insider data found for symbol {symbol}")
                            symbols_with_data.append(symbol)
                        else:
                            print(f"No insider transactions data parsed for symbol {symbol}")
                            symbols_without_data.append(symbol)
                    else:
                        print(f"No insider forms (3, 4, 5) found for symbol {symbol}")
                        symbols_without_data.append(symbol)
                else:
                    print(f"No filings found for symbol {symbol}")
                    symbols_without_data.append(symbol)
            except Exception as e:
                print(f"Error fetching data for symbol {symbol}: {e}")
                symbols_without_data.append(symbol)

            requests_made += 1

            # Implement rate limiting
            if requests_made % rate_limit_per_second == 0:
                time.sleep(1)

        print("\nData fetching completed.")
    except KeyboardInterrupt:
        print("\nProcess interrupted by user. Saving data collected so far...")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Saving data collected so far...")
    return insider_data, symbols_with_data, symbols_without_data

# Main script execution
if __name__ == '__main__':
    # Get the mapping of symbols to CIKs
    cik_map = get_cik_map()

    # For testing, you can use a subset of symbols (e.g., first 10)
    # subset_cik_map = dict(list(cik_map.items())[:10])
    # For full run, use cik_map
    # insider_transactions, symbols_with_data, symbols_without_data = fetch_insider_transactions(subset_cik_map)

    # Fetch insider transactions for all symbols
    insider_transactions, symbols_with_data, symbols_without_data = fetch_insider_transactions(cik_map)

    # Save to Excel
    if insider_transactions:
        df = pd.DataFrame(insider_transactions)
        excel_file_name = 'insider_transactions.xlsx'
        df.to_excel(excel_file_name, index=False)
        files.download(excel_file_name)
        print(f"Data saved to {excel_file_name} and downloaded to your local computer.")
    else:
        print("No insider transactions data collected.")

    # Optionally, save the lists of symbols with and without data
    with open('symbols_with_data.txt', 'w') as f:
        for symbol in symbols_with_data:
            f.write(f"{symbol}\n")
    with open('symbols_without_data.txt', 'w') as f:
        for symbol in symbols_without_data:
            f.write(f"{symbol}\n")

    # Download the symbol lists
    files.download('symbols_with_data.txt')
    files.download('symbols_without_data.txt')


  0%|          | 0/10077 [00:00<?, ?it/s]


Processing symbol AAPL (1/10077)
Fetching index page: https://www.sec.gov/Archives/edgar/data/320193/000032019324000116/0000320193-24-000116-index.html
Found XML file link: https://www.sec.gov/Archives/edgar/data/320193/000032019324000116/xslF345X05/wk-form4_1729204211.xml
Fetching XML file: https://www.sec.gov/Archives/edgar/data/320193/000032019324000116/xslF345X05/wk-form4_1729204211.xml
No reporting owner found
No non-derivative transactions found in the filing
Fetching index page: https://www.sec.gov/Archives/edgar/data/320193/000032019324000114/0000320193-24-000114-index.html
Found XML file link: https://www.sec.gov/Archives/edgar/data/320193/000032019324000114/xslF345X05/wk-form4_1728426607.xml
Fetching XML file: https://www.sec.gov/Archives/edgar/data/320193/000032019324000114/xslF345X05/wk-form4_1728426607.xml
No reporting owner found
No non-derivative transactions found in the filing
Fetching index page: https://www.sec.gov/Archives/edgar/data/320193/000032019324000112/00003

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

working fantastic missing ozk insider transactions

---



In [ ]:
# Install required libraries
!pip install finnhub-python openpyxl

import finnhub
import pandas as pd
import time
from google.colab import files
import requests

# Replace 'YOUR_API_KEY' with your actual Finnhub API key
API_KEY = 'csle2g1r01qq49fgr7lgcsle2g1r01qq49fgr7m0'

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key=API_KEY)


# Function to fetch insider transactions with rate limiting
def fetch_insider_transactions(symbols, rate_limit_per_minute=60):
    insider_data = []
    start_time = time.time()
    requests_made = 0
    for idx, symbol in enumerate(symbols):
        try:
            # Fetch insider transactions with '_from' date
            data = finnhub_client.stock_insider_transactions(symbol, _from='2020-01-01')
            # Check if data is available
            if 'data' in data and data['data']:
                for transaction in data['data']:
                    insider_data.append(transaction)
            else:
                print(f"No insider transactions for symbol {symbol}")
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                print("Rate limit exceeded. Sleeping for a minute.")
                time.sleep(60)
            else:
                print(f"HTTP error occurred for symbol {symbol}: {http_err}")
        except Exception as e:
            print(f"Error fetching data for symbol {symbol}: {e}")

        requests_made += 1
        # Implement rate limiting
        if requests_made >= rate_limit_per_minute:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            start_time = time.time()
            requests_made = 0
        else:
            time.sleep(1)  # Sleep for 1 second between requests

        # Progress update
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1}/{len(symbols)} symbols")
    return insider_data

# Initialize insider_transactions
insider_transactions = []

try:
    # Get list of US stock symbols
    symbols_data = finnhub_client.stock_symbols('US')
    symbol_list = [item['symbol'] for item in symbols_data]

    # Fetch insider transactions
    insider_transactions = fetch_insider_transactions(symbol_list)

except KeyboardInterrupt:
    print("\nProcess interrupted by user. Saving data collected so far...")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Saving data collected so far...")

finally:
    if insider_transactions:
        # Convert to DataFrame
        df = pd.DataFrame(insider_transactions)

        # Save to Excel file
        excel_file_name = 'insider_transactions.xlsx'
        df.to_excel(excel_file_name, index=False)

        # Download the Excel file to your local computer
        files.download(excel_file_name)
        print(f"Data saved to {excel_file_name} and downloaded to your local computer.")
    else:
        print("No data collected to save.")


{'data': [], 'symbol': 'OZK'}
No insider transactions for symbol MUHGF
No insider transactions for symbol GLXG
No insider transactions for symbol ALAI
No insider transactions for symbol OSGBS
No insider transactions for symbol CMTX
No insider transactions for symbol NRRMF
No insider transactions for symbol GSFP
No insider transactions for symbol HYBI
No insider transactions for symbol ZOOZ
No insider transactions for symbol XPNGF
No insider transactions for symbol GSDT
No insider transactions for symbol MIFTF
No insider transactions for symbol VSOGF
No insider transactions for symbol CCMMF
No insider transactions for symbol PCGR
No insider transactions for symbol BWZ
No insider transactions for symbol FTCFF
No insider transactions for symbol RSLOS
No insider transactions for symbol CINGF
No insider transactions for symbol PGAI
No insider transactions for symbol LIAF
No insider transactions for symbol ML.WS
No insider transactions for symbol HHDS
No insider transactions for symbol IGPG
